In [ ]:
from dbnet.trainer import SimpleTrainer
from dbnet.dataset import check

In [ ]:
!ls /datasets/almex_insurance/img | wc -l



# Instantiate a trainer

To override parameters in your model config (`db192mini.yaml` in this demo), just feed it to constructor of SimpleTrainer.
We only override the static input shape in `db192.yaml` with a dynamic one for demonstration purpose. This is unnecessary in practice since `SimpleTrainer` always force a dynamic input shape for training.

In [ ]:
# parameters to override
override = {
    "input_shape": (None, None, 3),
}

trainer = SimpleTrainer(conf_path="./conf/lin.yaml", **override)
# trainer = SimpleTrainer(conf_path="./conf/lin.yaml")

In [ ]:
trainer

## Check configuration of our trainer:

In [ ]:
trainer.model.summary()

## Login if using wandb

In [ ]:
if "WandbCallback" in trainer.train_conf["callbacks"]:
    !wandb login

# Check data pipelines

In [ ]:
# for idx, (x, y) in enumerate(trainer.ds_train.take(500)): print(f"\r{idx}", end="")
# for x, y in trainer.ds_val: continue
model.output


In [ ]:
# for x_train, y_train in trainer.ds_train: break
# for x_val, y_val in trainer.ds_val: break
check(x_train, y_train, figsize=(20, 10))
check(x_val, y_val, figsize=(20, 10))

# Train

To override parameters of callbacks and fit in your train config (`train.yaml` in this demo), just feed it to `SimpleTrainer.train`. For demonstration, we are override the original # of epochs (30) with 31.

**Notice: The current overriding mechanism uses a very naive implementation, thus it cannot work correctly when there are multiple subfields with the same name. For details, refer to `BaseTrainer._override`.**

In [ ]:
override = {"epochs": 31}
trainer.train(wandb_runname="demo", **override)

# Inference

In [ ]:
from tensorflow import keras
keras.backend.clear_session()

import matplotlib.pyplot as plt
import cv2
import numpy as np
from dbnet.model import dbnet_fpn, infer_strip
from dbnet.utils import load_conf

In [ ]:
conf = load_conf("./conf/db192mini.yaml")
conf

In [ ]:
conf = load_conf("./conf/db192mini.yaml")
conf["options"]["input_shape"] = (224, 224, 3)
model = dbnet_fpn(**conf["options"], batch_size=1, dynamic_shape=True)
# model = dbnet_fpn(**conf["options"], batch_size=1, dynamic_shape=False)
# release/ckpt/db192mini/2021_04_05_03_17_38/21-1.113.ckpt.index
# release/ckpt/db192mini/2021_04_12_08_03_09/20-0.440.ckpt.data-00000-of-00001
# workspace/dbnet.release.lin/ckpt/db192mini/2021_08_06_07_55_20/26-0.427.ckpt.index
model.load_weights("./ckpt/db192mini/2021_08_06_07_55_20/26-0.427.ckpt")
model = infer_strip(model)


In [ ]:
model.input

In [ ]:
# model.load_weights("./ckpt/mynum-mobilev3-1.0-192-min-imagenet/11-0.581.ckpt")

In [ ]:
# img1 = plt.imread("/workspace/release/106.jpg")

In [ ]:
# img = plt.imread("./data/test_insurance.jpg")
# img = plt.imread("/datasets/e2e/train/img_1.jpg")
# workspace/dbnet.release.lin/photo_2021-06-21_10-37-15.jpg
# workspace/dbnet.release.lin/photo_2021-06-21_17-41-11.jpg
img = plt.imread("photo_2021-06-21_17-41-11.jpg")
print(img.shape)

img = cv2.resize(img, (int(640 / img.shape[0] * img.shape[1] // 32 * 32), 640), cv2.INTER_AREA)

preds = model.predict(img[np.newaxis, ...].astype(np.float32))

print(preds[0].shape)
prob = preds[0][0, ..., 0]

plt.figure(figsize=(10, 10))
plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(prob, alpha=0.5)
plt.show()
# plt.imshow(plt.imread('./data/prob.jpg'))
# plt.show()


In [ ]:
new_model = keras.models.Model(model.get_layer('image').input,model.get_layer('binary').input)

In [ ]:
# model.get_layer('image').input

In [ ]:
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
model.save('my_model')

In [ ]:
keras.backend.clear_session()

In [ ]:
model1 = keras.models.load_model('my_model')

In [ ]:
model1.output

In [ ]:
# img = plt.imread("photo_2021-06-21_17-41-11.jpg")
print(img.shape)

img = cv2.resize(img, (int(640 / img.shape[0] * img.shape[1] // 32 * 32), 640), cv2.INTER_AREA)
print(img.shape)

preds = model1.predict(img[np.newaxis, ...].astype(np.float32))
print(preds.shape)
prob = preds[0, ..., 0]                                                                                                                                                        

plt.figure(figsize=(10, 10))
plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(prob, alpha=0.5)
plt.show()

In [ ]:
# new_model = keras.models.Model(model.get_layer('image').input,model.get_layer('threshold').input)
img = plt.imread("photo_2021-06-21_17-41-11.jpg")
print(img.shape)

img = cv2.resize(img, (int(640 / img.shape[0] * img.shape[1] // 32 * 32), 640), cv2.INTER_AREA)
print(img.shape)

preds = model.predict(img[np.newaxis, ...].astype(np.float32))
print(preds[0].shape)
prob = preds[0][0, ..., 0]
th = preds[1][0, ..., 0]
binary = preds[2][0, ..., 0]

plt.figure(figsize=(10, 10))
# plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(prob, alpha=0.5)
plt.show()


plt.figure(figsize=(10, 10))
# plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(th, alpha=0.5)
plt.show()


plt.figure(figsize=(10, 10))
# plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(binary, alpha=0.5)
plt.show()

In [ ]:
model.output_shape

In [ ]:
model = trainer.model
# workspace/dbnet.release.lin/ckpt/db192mini/2021_08_06_07_55_20/26-0.427.ckpt.data-00000-of-00001
model.load_weights('./ckpt/db192mini/2021_08_06_07_55_20/26-0.427.ckpt')

In [ ]:
model.summary()

In [ ]:
new_model = keras.models.Model(model.get_layer('image').input,model.get_layer('conv2d_transpose_9').output)

In [ ]:
img = plt.imread("photo_2021-06-21_17-41-11.jpg")
print(img.shape)

img = cv2.resize(img, (int(640 / img.shape[0] * img.shape[1] // 32 * 32), 640), cv2.INTER_AREA)
print(img.shape)

preds = new_model.predict(img[np.newaxis, ...].astype(np.float32))
print(preds[0].shape)
prob = preds[0]
# th = preds[1][0, ..., 0]
# binary = preds[2][0, ..., 0]

plt.figure(figsize=(10, 10))
# plt.imshow(img, alpha=0.9)
# plt.show()
plt.imshow(np.exp(prob)/(1+np.exp(prob)), alpha=0.5)
plt.show()

In [ ]:
new_model.save('output_transpose')